## 소방청 데이터

https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword=

일반현황 > 전국소방기관 안내 > '서울' 검색

페이지 바꾸는 것은 셀레늄으로, 바꾼 후에는 BS로 크롤링
관서명 주소 전화번호 > csv파일로 저장
100건 가져오기

kakao api 로 위도 경도 값 받아옴
(못받아온 경우 인덱스 확인 후 drop)

지도에 표시

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm

import numpy as np

from urllib.parse import quote
import folium, json

### 데이터 크롤링

In [ ]:
https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword=
https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=37.5382842&myY=126.9213649&searchKeyword=%EC%84%9C%EC%9A%B8&pageIndex=1
https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=37.5382842&myY=126.9213649&searchKeyword=%EC%84%9C%EC%9A%B8&pageIndex=10

In [6]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword='
seoul = '%EC%84%9C%EC%9A%B8'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

In [7]:
inputElement = driver.find_element(By.CSS_SELECTOR, '#searchKeyword')
inputElement.send_keys('서울')
time.sleep(2)

In [8]:
driver.find_element(By.CSS_SELECTOR, '#fsSearchBtn').click()
time.sleep(2)

In [48]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [54]:
list = soup.select('.stations-list>li')
len(list)

10

In [61]:
li = list[0]
li

<li id="0" style="height: 172px;">
<img class="marker" src="/resources/img/firestation/map_ic_station.png"/>
<span class="title">동작소방서</span>
<address>서울특별시 동작구 여의대방로16길 55(신대방동)</address>
<span class="tel">02-847-1190</span>
<span class="info">서울소방재난본부</span>
</li>

In [72]:
li.select('.title')[0].text
li.select('address')[0].text
li.select('.tel')[0].text

'02-847-1190'

In [73]:
lines = []
for li in list:
    title = li.select('.title')[0].text
    addr = li.select('address')[0].text
    tel = li.select('.tel')[0].text
    lines.append([title, addr, tel])
lines

[['동작소방서', '서울특별시 동작구 여의대방로16길 55(신대방동)', '02-847-1190'],
 ['서대문소방서', '서울특별시 서대문구 연희로 182(연희동)', '02-3144-1190'],
 ['광진소방서', '서울특별시 광진구 광나루로 480(구의동)', '02-457-0119'],
 ['송파소방서', '서울특별시 송파구 오금로51길 56(마천동)', '02-403-2119'],
 ['양천소방서', '서울특별시 양천구 목동서로 180(목동)', '02-2655-1119'],
 ['은평소방서', '서울특별시 은평구 통일로 962(진관동)', '02-355-0119'],
 ['노원소방서', '서울특별시 노원구 한글비석로 1길 8(하계동)', '02-977-0119'],
 ['종로소방서', '서울특별시 종로구 종로1길 28(수송동)', '02-735-6119'],
 ['도봉소방서', '서울특별시 도봉구 도봉로 666(방학동)', '02-3492-3437'],
 ['용산소방서', '서울특별시 용산구 한강대로 167(한강로2가)', '02-794-0119']]

In [75]:
df = pd.DataFrame(lines, columns = ['관서명','주소','전화번호'])
df         

,관서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119
5,은평소방서,서울특별시 은평구 통일로 962(진관동),02-355-0119
6,노원소방서,서울특별시 노원구 한글비석로 1길 8(하계동),02-977-0119
7,종로소방서,서울특별시 종로구 종로1길 28(수송동),02-735-6119
8,도봉소방서,서울특별시 도봉구 도봉로 666(방학동),02-3492-3437
9,용산소방서,서울특별시 용산구 한강대로 167(한강로2가),02-794-0119


In [76]:
lines = []
url_page = url
for i in tqdm(range(1,11)):
    url_page = url + seoul + '&pageindex=' + str(i)
    driver.get(url_page)    # 크롤링사이트 호출하기
    html = driver.page_source    # 브라우저에 보이는 그대로의 html을 가져옴
    soup = BeautifulSoup(html, 'html.parser')  
    
    for li in list:
        title = li.select('.title')[0].text
        addr = li.select('address')[0].text
        tel = li.select('.tel')[0].text
        lines.append([title, addr, tel])
df = pd.DataFrame(lines, columns = ['관서명','주소','전화번호'])
df         

  0%|          | 0/10 [00:00<?, ?it/s]

,관서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119
...,...,...,...
95,은평소방서,서울특별시 은평구 통일로 962(진관동),02-355-0119
96,노원소방서,서울특별시 노원구 한글비석로 1길 8(하계동),02-977-0119
97,종로소방서,서울특별시 종로구 종로1길 28(수송동),02-735-6119
98,도봉소방서,서울특별시 도봉구 도봉로 666(방학동),02-3492-3437


In [77]:
# KAKAO API로 위도 경도값을 추춘
with open('./kakao_API.txt') as file:
    kakao_key = file.read()

In [79]:
search_url = "https://dapi.kakao.com/v2/local/search/address.json"
url = f'{search_url}?query={quote(df.주소[i])}'    #query를 보낼 때는 ?로 연결됨
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EB%8F%99%EC%9E%91%EA%B5%AC%20%EC%97%AC%EC%9D%98%EB%8C%80%EB%B0%A9%EB%A1%9C16%EA%B8%B8%2055%28%EC%8B%A0%EB%8C%80%EB%B0%A9%EB%8F%99%29'

In [81]:
lat_list = []
lng_list = []
for i in df.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.주소[i])}'  
    result = requests.get(url, headers = {"Authorization": f'KakaoAK {kakao_key}'}).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:   # 행정동명으로 표기되어 있지 않아서 못 받아오는 경우
        print(df.관서명[i])

In [83]:
df['위도'] = lat_list
df['경도'] = lng_list

In [85]:
df.head()

,관서명,주소,전화번호,위도,경도
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,37.494672,126.917719
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,37.573205,126.935996
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119,37.544826,127.082779
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119,37.499833,127.142592
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119,37.530236,126.872342


In [87]:
# 마커표시

map = folium.Map(location = [37.5502, 126.982], zoom_start = 12)
for i in df.index:
    folium.Marker(location = [df.위도[i], df.경도[i]],
                 popup = folium.Popup(df.주소[i], max_width = 200),
                 tooltip = f'{df.관서명[i]}, {df.전화번호[i]}'
                 ).add_to(map)
map

# tch

In [112]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import math
import pandas as pd
from tqdm.notebook import tqdm

In [89]:
driver = webdriver.Chrome()
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword='
driver.get(url)

In [90]:
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [91]:
sp = BeautifulSoup(driver.page_source,'html.parser')  # html가져오기

In [92]:
lis = sp.select('.stations-list > li')
len(lis)

10

In [94]:
li = lis[0]
li

<li id="0" style="height: 172px;">
<img class="marker" src="/resources/img/firestation/map_ic_station.png"/>
<span class="title">동작소방서</span>
<address>서울특별시 동작구 여의대방로16길 55(신대방동)</address>
<span class="tel">02-847-1190</span>
<span class="info">서울소방재난본부</span>
</li>

In [95]:
name = li.select_one('.title').get_text().strip()
addr = li.find('address').get_text().strip()
tel = li.select_one('.tel').get_text().strip()
name, addr, tel

('동작소방서', '서울특별시 동작구 여의대방로16길 55(신대방동)', '02-847-1190')

In [96]:
lins = []
for li in lis:
    name = li.select_one('.title').get_text().strip()
    addr = li.find('address').get_text().strip()
    tel = li.select_one('.tel').get_text().strip()
    lins.append([name, addr, tel])

In [110]:
# 검색결과 건수 
num = int(sp.select('.stations-local-after > p > strong')[1].get_text().replace('건',''))
num

100

In [101]:
### 페이지 넘기기
# 1페이지에서 다음페이지를 누르면 2페이지가 아니고, 바로 3페이지가 나옴
# 3페이지로 넘어가는 부분을 if문으로 처리

driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[5]/a').click()
# 홀수페이지만 출력됨

# 짝수페이지는 별도로 출력
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()



### 전체 코드 정리

In [156]:
driver = webdriver.Chrome()
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword='
driver.get(url)
time.sleep(2)
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()
time.sleep(2)

In [164]:
driver = webdriver.Chrome()
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword='
driver.get(url)
time.sleep(2)
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()
time.sleep(2)

num = int(sp.select('.stations-local-after > p > strong')[1].get_text().replace('건','').strip())
count = math.ceil(num/10)   # math.ceil: 나눴을 때 무조건 1을 더해주는 함수

lins = []
for i in tqdm(range(count)):
    if i >=1 and i % 2 != 1 : 
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[5]/a').click()
        time.sleep(3)
    if i >=1 and i % 2 != 0 : 
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(3)
    sp = BeautifulSoup(driver.page_source, 'html.parser')
    lis = sp.select('.stations-list > li')
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.find('address').get_text().strip()
        tel = li.select_one('.tel').get_text().strip()
        lins.append([name, addr, tel])

  0%|          | 0/10 [00:00<?, ?it/s]

In [165]:
len(lins)

100

In [166]:
driver.close()

In [170]:
df = pd.DataFrame(lins, columns = ['관서명','주소','전화번호'])
df.to_csv('data1/서울소방서.csv', index = False)  # 저장

## 데이터 분석 및 시각화

In [171]:
df = pd.read_csv('data1/서울소방서.csv')
df.head()

,관서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119


In [172]:
df['관서명'].nunique()

100

In [173]:
# KAKAO API로 위도 경도값 추출
with open('./kakao_API.txt') as file:
    kakao_key = file.read()

search_url = "https://dapi.kakao.com/v2/local/search/address.json"
url = f'{search_url}?query={quote(df.주소[i])}'    #query를 보낼 때는 ?로 연결됨

lat_list = []
lng_list = []
for i in df.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.주소[i])}'  
    result = requests.get(url, headers = {"Authorization": f'KakaoAK {kakao_key}'}).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:   # 행정동명으로 표기되어 있지 않아서 못 받아오는 경우
        print(df.관서명[i])

은평소방서-수색-119 안전센터


In [174]:
len(lat_list), len(lng_list)

(99, 99)

In [176]:
df[df.관서명.isin(['은평소방서-수색-119 안전센터'])].주소

30    서울특별시 은평구 수색로 294 (수색동)
Name: 주소, dtype: object

In [177]:
df.drop(30, axis = 0, inplace = True)

In [178]:
df['위도'] = lat_list
df['경도'] = lng_list

In [187]:
# 마커표시

map = folium.Map(location = [37.5502, 126.982], zoom_start = 12)
for i in df.index:
    folium.Marker(location = [df.위도[i], df.경도[i]],
                 popup = folium.Popup(df.주소[i], max_width = 200),
                 tooltip = f'{df.관서명[i]}\n{df.전화번호[i]}',
                  icon=folium.Icon(icon="glyphicon glyphicon-home")
                 ).add_to(map)
title_html = '<h3 align = "center" style="font-size:20px"> 서울소재 소방서 위치 </h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

In [190]:
# 빠진 은평소방서수색119안전센터 넣고 싶으면, 주소 다시 찾아서 위/경도값 삽입

url_320 = f'{search_url}?query={quote("서울특별시 은평구 수색로 320")}'
result = requests.get(url_320, headers = {"Authorization": f'KakaoAK {kakao_key}'}).json()
print(float(result['documents'][0]['x']))
print(float(result['documents'][0]['y']))

126.889905679045
37.5852677029786
